# **Modeling Logical Constraints Involving Binary Variables**#


**What will you learn?**

*   Modeling - Knapsack problem
*   Implementation using GurobiPy (python)
*   Logical constraints and their implementation
*   Standard modeling interview questions (logical constraints involving binary variables)


# **Knapsack Problem:**
Given a stack of 500 shipments with their values, weights and volumes, and a truck container with weight capacity of 20500 Kg and load volume of 3,000 cu. ft., write an optimization model to find the set of shipments which maximize the value that can be packed in the container, while respecting the weight and volume constraints.


# **Mathematical Model:**
**Parameters:**

\begin{align*}
N &= \text{number of shipments } \\
    value_i &= \text{value of shipment }i, (\text{\$}) \\
    weight_i &= \text{weight of shipment }i, (\text{kg}) \\
    volume_i &= \text{volume of shipment }i, (\text{cu. ft.}) \\
    weight^{max} &= \text{maximum carrying weight of the container}, (\text{kg}) \\
        volume^{max} &= \text{maximum carrying capacity (volume) of the container}, (\text{cu. ft.}) \\
\end{align*}
**Decision variables:**
  \begin{equation}
    x_i =
    \begin{cases}
      1, & \text{if shipment }i \text{ is seleceted} \\
      0, & \text{otherwise}
    \end{cases}
  \end{equation}

**Objective function:**
$$ \textrm{Maximize} \sum \limits _{i=1} ^{N} x_i \cdot value_i $$

**Constraints:**


1.   Total weight of all selected shipments can not be more than the maximum weight capacity of the container.
$$  \sum \limits _{i=1} ^{N} x_i \cdot weight_i \leq weight^{max}$$
2.   Total volume of all selected shipments can not be more than the maximum volume of the container.
$$  \sum \limits _{i=1} ^{N} x_i \cdot volume_i \leq volume^{max}$$





# **Import all required python packages:**

In [ ]:
import numpy as np
from time import gmtime, strftime
np.random.seed(1729) # setting random seed to get the same output in each run since I am generating random inputs
# Display current time
print('Date & Time now: ', strftime("%d-%m-%Y %H:%M:%S %Z%z", gmtime()))

Date & Time now:  17-07-2021 14:07:49 GMT+0000


# **Data Generation:**

In [ ]:
number_of_shipments = 500
weights_mu, weights_sigma = 70, 20 # weights of the shipments are randomly generated N(weights_mu, weights_sigma)
weights = abs(np.around(np.random.normal(weights_mu, weights_sigma, number_of_shipments),2)) # taken absolute values since weights cannot be negetive, rounded to second decimal places
volumes_mu, volumes_sigma = 10, 2 # volumes of the shipments are randomly generated N(volumes_mu, volumes_sigma)
volumes = abs(np.around(np.random.normal(volumes_mu, volumes_sigma, number_of_shipments),2)) # taken absolute values since weights cannot be negetive, rounded to second decimal places
values_mu, values_sigma = 50, 30 # values of the shipments are randomly generated N(values_mu, values_sigma)
values = abs(np.around(np.random.normal(values_mu, values_sigma, number_of_shipments),2)) # taken absolute since values of a shipment cannot be negetive, rounded to second decimal places
maximum_weight, maximum_volume = 20500, 3000 # maximum weight and volume capacity of the container

# **Model implementation using GurobiPy:**
*Free Gurobi solver limitation on problem size* - Problems with at most 2000 variables and 2000 constraints.


In [ ]:
# Install guribipy and import to use gurobi as a solver
%pip install -i https://pypi.gurobi.com gurobipy
import gurobipy as gp
from gurobipy import *

# Declare and Initialize the model
optimization_model = gp.Model('Knapsack Problem')

# define decision variables - x_i is 1 if shipment 'i' is selected; 0 otherwise. (Binary variables)
x = optimization_model.addVars(number_of_shipments, vtype=GRB.BINARY, name = 'shipment')

# define objective (maximize total values of all selected shipments)
optimization_model.setObjective(gp.quicksum(x[i] * values[i] for i in range(number_of_shipments)), GRB.MAXIMIZE)

# add the maximum weight constraint
optimization_model.addConstr(gp.quicksum(weights[i] * x[i] for i in range(number_of_shipments)) <= maximum_weight, name = 'weight_capacity_constraint')

# add the maximum volume constraint
optimization_model.addConstr(gp.quicksum(volumes[i] * x[i] for i in range(number_of_shipments)) <= maximum_volume, name = 'volume_capacity_constraint')

# solve the model using Gurobi solver
optimization_model.setParam('LogToConsole', 0) # To supress the solver log in the output console
optimization_model.optimize()
optimization_model.write('Knapsack_Problem_Base_Case.lp')

# identify the model status (optimal, infeasible, unbounded)
if optimization_model.status == GRB.OPTIMAL:
  optimal_objective_value = optimization_model.objVal # obtain the optimal objective value for future use
  print('\nThe optimal objective: %g' % optimal_objective_value)
elif optimization_model.status == GRB.INF_OR_UNBD:
  print('Model is infeasible or unbounded')
  sys.exit(0)
elif optimization_model.status == GRB.INFEASIBLE:
  print('Model is infeasible')
  sys.exit(0)
elif optimization_model.status == GRB.UNBOUNDED:
  print('Model is unbounded')
  sys.exit(0)
else:
  print('Optimization ended with status %d' % optimization_model.status)
  sys.exit(0)

# selected shipments
selected_shipments = [] #initialization [shipment_id, value, weight, volume]
not_selected_shipments = [] #initialization [shipment_id, value, weight, volume]
total_weight_of_selected_items = 0  #initialization
total_volume_of_selected_items = 0  #initialization
for variables in optimization_model.getVars():
  if variables.x > 0.5:
    selected_shipments.append([variables.index, values[variables.index], weights[variables.index], volumes[variables.index]])
    total_weight_of_selected_items += weights[variables.index]
    total_volume_of_selected_items += volumes[variables.index]
  else:
    not_selected_shipments.append([variables.index, values[variables.index], weights[variables.index], volumes[variables.index]])
selected_shipments = sorted(selected_shipments, key=lambda x: x[1], reverse=True) # Shipments are sorted based on values of selected items, descending order
not_selected_shipments = sorted(not_selected_shipments, key=lambda x: x[1], reverse=True) # Shipments are sorted based on values of not selected items, descending order
print('First twenty selected shipments with highest values: ', selected_shipments[0:20])
print('First twenty not selected shipments: ', not_selected_shipments[0:20])
print('Total number of shipments in the container: ', len(selected_shipments))
print('Total weight in the container: ', round(total_weight_of_selected_items,2), ' kg')
print('Total volume in the container: ', round(total_volume_of_selected_items, 2), ' cu. ft.')



Looking in indexes: https://pypi.gurobi.com
     |████████████████████████████████| 11.1MB 240kB/s 
Restricted license - for non-production use only - expires 2022-01-13

The optimal objective: 20734.2
First twenty selected shipments with highest values:  [[205, 129.28, 58.28, 12.57], [326, 125.33, 93.87, 9.02], [123, 123.17, 88.53, 8.78], [137, 120.23, 71.49, 9.24], [122, 119.13, 87.59, 9.04], [221, 117.73, 60.26, 6.89], [45, 112.09, 78.18, 8.68], [306, 111.66, 61.36, 12.31], [426, 109.96, 31.21, 9.06], [39, 109.18, 69.61, 9.9], [450, 107.16, 81.76, 11.07], [34, 106.66, 92.67, 12.14], [352, 106.35, 66.54, 11.0], [187, 106.15, 74.65, 9.71], [466, 105.39, 54.94, 7.95], [233, 105.34, 61.25, 10.24], [293, 104.64, 72.43, 10.22], [185, 103.44, 71.17, 12.46], [474, 102.93, 65.36, 6.87], [173, 102.84, 81.87, 11.17]]
First twenty not selected shipments:  [[279, 48.68, 84.44, 12.85], [366, 46.22, 80.92, 12.35], [339, 45.73, 71.34, 11.94], [356, 45.43, 39.05, 16.71], [245, 44.44, 87.15, 12.26], 

# **Logical constraints:**


1.   If shipment **A** is selected then shipment **B** must be selected.

  *Constraint 1:*  $ x_B \geq x_A $
2.   Shipments **C** and **D** cannot be selected together.

  *Constraint 2:*  $  x_C + x_D \leq 1 $
3.   Shipment **G** must be selected if both shipment **E** and **F** are selected.

  *Constraint 3:*  $  x_G \geq  x_E + x_F - 1 $
4.   Shipments **I**, **J**, \& **K** cannot be selected together.  
  *Constraint 4:*  $  x_I +  x_J + x_K \leq 2 $

5.   Shipments **L** \& **M** can be selected together only if shipments **O** \& **P** are selected.

  *Constraint 5:*  $  x_L +  x_M \leq  x_O + x_P$

6. Shipments **Q** is selected if and only if **R** is seleced.

  *Constraint 6:*  $  x_Q =  x_R$

7. Must select shipment **S** or shipment **T** or both.

  *Constraint 7:*  $  x_S + x_T \geq 1 $

In [ ]:
# First twenty selected shipments [Shipment_ID, Value, Weight, Volumw]:  [[205, 129.28, 58.28, 12.57], [326, 125.33, 93.87, 9.02], [123, 123.17, 88.53, 8.78], [137, 120.23, 71.49, 9.24], [122, 119.13, 87.59, 9.04], [221, 117.73, 60.26, 6.89], [45, 112.09, 78.18, 8.68], [306, 111.66, 61.36, 12.31], [426, 109.96, 31.21, 9.06], [39, 109.18, 69.61, 9.9], [450, 107.16, 81.76, 11.07], [34, 106.66, 92.67, 12.14], [352, 106.35, 66.54, 11.0], [187, 106.15, 74.65, 9.71], [466, 105.39, 54.94, 7.95], [233, 105.34, 61.25, 10.24], [293, 104.64, 72.43, 10.22], [185, 103.44, 71.17, 12.46], [474, 102.93, 65.36, 6.87], [173, 102.84, 81.87, 11.17]]

# First ten not selected shipments:  [[279, 48.68, 84.44, 12.85], [366, 46.22, 80.92, 12.35], [339, 45.73, 71.34, 11.94], [356, 45.43, 39.05, 16.71], [245, 44.44, 87.15, 12.26], [465, 43.99, 105.14, 8.84], [418, 43.89, 78.77, 11.47], [476, 43.32, 87.13, 10.38], [231, 42.87, 81.31, 11.17], [158, 42.66, 68.05, 10.59]]

# add constraint 1: If shipment 205 is selected then shipment 279 must be selected. Currently, shipment 279 is not selected and shipments 205 is selected. Let's model this constraint and observe the result.
optimization_model.addConstr(x[279] >= x[205], name = 'constraint_1')

# add constraint 2: Shipments 326 and 123 cannot be selected together. Currently, both shipments are selected. Let's model this constraint and observe the result.
optimization_model.addConstr(x[326] + x[123] <= 1, name = 'constraint_2')

# add constraint 3: Shipment 339 must be selected if both shipment 205 and 326 are selected. Currently, both shipments 205 and 326 are selected and 339 is not selected. Let's model this constraint and observe the result.
optimization_model.addConstr(x[339] >= x[205] + x[326] - 1, name = 'constraint_3')

# add constraint 4: Shipments 221, 45, & 306 cannot be selected together. Currently, all these shipments are selected. Let's model this constraint and observe the result.
optimization_model.addConstr(x[221] + x[45] + x[306] <= 2, name = 'constraint_4')

# add constraint 5: Shipments 352 & 187 can be selected together only if shipments 339 & 356 are selected. Currently, shipments 352 and 187 are selected and shipments 339 and 35 are not selected. Let's model this constraint and observe the result.
optimization_model.addConstr(x[352] + x[187] <= x[339] + x[356], name = 'constraint_5')

# add constraint 6: Shipments 245 is selected if and only if 205 is seleced. Currently, shipment 205 is selected and shipment 245 is not selected. Let's model this constraint and observe the result.
optimization_model.addConstr(x[245]  == x[205], name = 'constraint_6')

# add constraint 7: Must select shipment 279 or shipment 366 or both. Currently, both shipments 279 & 366 are not selected. Let's model this constraint and observe the result.
optimization_model.addConstr(x[279] + x[366] >= 1, name = 'constraint_7')

# solve the model using Gurobi solver
optimization_model.setParam('LogToConsole', 0) # To supress the solver log in the output console
optimization_model.setParam('LogFile', "logs_Knapsack_Problem_with_Logical_Constraints.txt")
optimization_model.optimize()
optimization_model.write('Knapsack_Problem_with_Logical_Constraints.lp')

# identify the model status (optimal, infeasible, unbounded)
if optimization_model.status == GRB.OPTIMAL:
  optimal_objective_value = optimization_model.objVal # obtain the optimal objective value for future use
  print('\nThe optimal objective: %g' % optimal_objective_value)
elif optimization_model.status == GRB.INF_OR_UNBD:
  print('Model is infeasible or unbounded')
  sys.exit(0)
elif optimization_model.status == GRB.INFEASIBLE:
  print('Model is infeasible')
  sys.exit(0)
elif optimization_model.status == GRB.UNBOUNDED:
  print('Model is unbounded')
  sys.exit(0)
else:
  print('Optimization ended with status %d' % optimization_model.status)
  sys.exit(0)

# selected shipments
selected_shipments = [] # initialization [shipment_id, value, weight, volume]
not_selected_shipments = [] # initialization [shipment_id, value, weight, volume]
selected_shipment_IDs = [] # initialization
total_weight_of_selected_items = 0  # initialization
total_volume_of_selected_items = 0  # initialization
for variables in optimization_model.getVars():
  if variables.x > 0.5:
    selected_shipments.append([variables.index, values[variables.index], weights[variables.index], volumes[variables.index]])
    selected_shipment_IDs.append(variables.index)
    total_weight_of_selected_items += weights[variables.index]
    total_volume_of_selected_items += volumes[variables.index]
  else:
    not_selected_shipments.append([variables.index, values[variables.index], weights[variables.index], volumes[variables.index]])
selected_shipments = sorted(selected_shipments, key=lambda x: x[1], reverse=True) # Shipments are sorted based on values of selected items, descending order
print('First twenty selected shipments with highest values: ', selected_shipments[0:20])
print('First twenty not selected shipments: ', not_selected_shipments[0:20])
print('Total number of shipments in the container: ', len(selected_shipments))
print('Total weight in the container: ', round(total_weight_of_selected_items,2), ' kg')
print('Total volume in the container: ', round(total_volume_of_selected_items, 2), ' cu. ft.')


The optimal objective: 20565.4
First twenty selected shipments with highest values:  [[205, 129.28, 58.28, 12.57], [326, 125.33, 93.87, 9.02], [137, 120.23, 71.49, 9.24], [122, 119.13, 87.59, 9.04], [221, 117.73, 60.26, 6.89], [45, 112.09, 78.18, 8.68], [426, 109.96, 31.21, 9.06], [39, 109.18, 69.61, 9.9], [450, 107.16, 81.76, 11.07], [34, 106.66, 92.67, 12.14], [352, 106.35, 66.54, 11.0], [187, 106.15, 74.65, 9.71], [466, 105.39, 54.94, 7.95], [233, 105.34, 61.25, 10.24], [293, 104.64, 72.43, 10.22], [185, 103.44, 71.17, 12.46], [474, 102.93, 65.36, 6.87], [173, 102.84, 81.87, 11.17], [178, 102.81, 95.9, 11.94], [241, 101.01, 79.17, 9.69]]
First twenty not selected shipments:  [[0, 20.76, 56.25, 10.61], [1, 22.22, 53.58, 10.86], [2, 20.33, 103.05, 9.57], [3, 39.76, 58.49, 11.11], [7, 23.81, 52.84, 10.31], [8, 23.04, 71.5, 12.67], [9, 10.24, 80.59, 9.02], [10, 26.62, 72.42, 13.28], [12, 39.59, 38.87, 12.91], [16, 16.71, 72.19, 7.6], [19, 27.41, 75.0, 9.94], [20, 5.31, 95.97, 10.82], [

## Some standard modeling interview questions involving binary variables $(x_1, x_2 \in \{0, 1\})$ : (try implementing these constraints)


*   **AND**:   $x_1 = 1 \wedge x_2 = 1 \implies x_1 + x_2 = 2$
*   **OR**:   $x_1 = 1 \vee x_2 = 1 \implies x_1 + x_2 \ge 1$
*   **Exclusive OR (not both)**:   $x_1 = 1 \oplus x_2 = 1 \implies x_1 + x_2 = 1$
*   **At least**: at least pick $k$ items   $\implies \sum_i x_i \ge k $
*   **At most**: at most pick $k$ items   $\implies \sum_i x_i \le k $
*   **Exact**: pick $k$ items   $\implies \sum_i x_i = k $
*   **If then (implication)**:  if $x_1 = 1$ then $x_2 = 1$ $\implies x_1 \leq x_2$



##I hope, you learned something today. Since you have come this far, **please share your feedback with [me](https://www.linkedin.com/in/alok7iitb/).**